I WILL PERFORM ALL MY ONE HOT ENCODING OF CATEGORICAL FEATURES BEFORE CROSS VALIDATION PIPELINE, BUT ALL NUMERICAL PREPROCESSING WILL BE PERFORMED WITHIN.

In [58]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
df = pd.read_csv('/home/jovyan/data1030/data1030-oscars-prediction-project/data/movies.csv')

# parsing and one hot encoding of "weird" columns, unique due to it's structure
bar_cats =['New_York_Film_Critics_Circle_won_categories','Hollywood_Film_won_categories','Hollywood_Film_nominated_categories','Austin_Film_Critics_Association_won_categories','Austin_Film_Critics_Association_nominated_categories','Denver_Film_Critics_Society_won_categories','Denver_Film_Critics_Society_nominated_categories','Boston_Society_of_Film_Critics_won_categories','Boston_Society_of_Film_Critics_nominated_categories','New_York_Film_Critics_Circle_nominated_categories','Los_Angeles_Film_Critics_Association_won_categories','Los_Angeles_Film_Critics_Association_nominated_categories','Online_Film_Critics_Society_won_categories','Online_Film_Critics_Society_nominated_categories','People_Choice_won_categories','People_Choice_nominated_categories','London_Critics_Circle_Film_won_categories','London_Critics_Circle_Film_nominated_categories','American_Cinema_Editors_won_categories','American_Cinema_Editors_nominated_categories','Costume_Designers_Guild_won_categories','Costume_Designers_Guild_nominated_categories','Online_Film_Television_Association_won_categories','Online_Film_Television_Association_nominated_categories','Producers_Guild_won_categories','Producers_Guild_nominated_categories','Art_Directors_Guild_won_categories','Art_Directors_Guild_nominated_categories','Writers_Guild_won_categories','Writers_Guild_nominated_categories','Critics_Choice_won_categories','Critics_Choice_nominated_categories','Directors_Guild_won_categories','Directors_Guild_nominated_categories','Screen_Actors_Guild_won_categories','Screen_Actors_Guild_nominated_categories','BAFTA_won_categories','BAFTA_nominated_categories','Golden_Globes_won_categories','Golden_Globes_nominated_categories','Oscar_nominated_categories','genre']
bar_ohe = []
for label in bar_cats:
    df[label].fillna("nan", inplace=True)
    cat_series = df[label].apply(lambda x: (x.split('|')))
    cats = [cat  for ele in list(cat_series) for cat in ele]
    cat_set = set(cats)
    if 'nan' in cat_set:
        cat_set.remove('nan')
    total_cats = list(cat_set)
    cat_cols = [label + ": " + x for x in total_cats]

    mv_vals = np.asarray(cat_series)
    cat_ohe = np.zeros((len(cat_series),len(total_cats)))
    for i in range(len(mv_vals)):
        for j in range(len(total_cats)):
            cat_ohe[i,j] = total_cats[j] in mv_vals[i]
    ohe = pd.DataFrame(cat_ohe, columns = cat_cols)
    bar_ohe.append(ohe)
bar_ohe = pd.concat(bar_ohe,axis=1)  

ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
oe = OrdinalEncoder(categories = ['No','Yes'])

#Replacing columns with "Yes"/"No" values to 1/0
ord_cols = 'Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated'
ord_cols = [x for x in ord_cols.split(',')]
for col in ord_cols:
    df[col].replace({'Yes':1,'No':0}, inplace=True)

ohe_data = pd.DataFrame(ohe.fit_transform(np.array(df['certificate'].replace({np.nan:"Missing"})).reshape(-1,1)))

ohe_data.columns= ohe.get_feature_names(['certificate'])

categorical_data = pd.concat([df[ord_cols],ohe_data,bar_ohe], axis=1)

#trying to figure out mcar
df_mcar = pd.concat([df[ord_cols],ohe_data],axis=1)

In [59]:
le = LabelEncoder()
le_data = pd.DataFrame(le.fit_transform(df['Oscar_Best_Picture_won']))
le_data.columns=['Oscar_Best_Picture_won']
y = le_data['Oscar_Best_Picture_won']
y.value_counts()/len(y)

0    0.985425
1    0.014575
Name: Oscar_Best_Picture_won, dtype: float64

In [60]:
print("Of the features that have missing values, the fraction of missing values is:")
nulls = df.isnull().sum(axis=0)/df.shape[0]
print(nulls[nulls > 0])
print("")
print("The total fraction of missing features in the data set is:")
print(sum(df.isnull().sum(axis=1)!=0)/df.shape[0])
print("")

Of the features that have missing values, the fraction of missing values is:
certificate                  0.021053
metascore                    0.023482
gross                        0.034008
release_date                 0.024291
user_reviews                 0.011336
critic_reviews               0.008097
popularity                   0.109312
release_date.year            0.024291
release_date.month           0.024291
release_date.day-of-month    0.024291
release_date.day-of-week     0.024291
dtype: float64

The total fraction of missing features in the data set is:
0.14251012145748987



In [61]:
drop_cols =['Oscar_Best_Picture_won','certificate','movie_id','synopsis','New_York_Film_Critics_Circle_won_categories','Hollywood_Film_won_categories','Hollywood_Film_nominated_categories','Austin_Film_Critics_Association_won_categories','Austin_Film_Critics_Association_nominated_categories','Denver_Film_Critics_Society_won_categories','Denver_Film_Critics_Society_nominated_categories','Boston_Society_of_Film_Critics_won_categories','Boston_Society_of_Film_Critics_nominated_categories','New_York_Film_Critics_Circle_nominated_categories','Los_Angeles_Film_Critics_Association_won_categories','Los_Angeles_Film_Critics_Association_nominated_categories','Online_Film_Critics_Society_won_categories','Online_Film_Critics_Society_nominated_categories','People_Choice_won_categories','People_Choice_nominated_categories','London_Critics_Circle_Film_won_categories','London_Critics_Circle_Film_nominated_categories','American_Cinema_Editors_won_categories','American_Cinema_Editors_nominated_categories','Costume_Designers_Guild_won_categories','Costume_Designers_Guild_nominated_categories','Online_Film_Television_Association_won_categories','Online_Film_Television_Association_nominated_categories','Producers_Guild_won_categories','Producers_Guild_nominated_categories','Art_Directors_Guild_won_categories','Art_Directors_Guild_nominated_categories','Writers_Guild_won_categories','Writers_Guild_nominated_categories','Critics_Choice_won_categories','Critics_Choice_nominated_categories','Directors_Guild_won_categories','Directors_Guild_nominated_categories','Screen_Actors_Guild_won_categories','Screen_Actors_Guild_nominated_categories','BAFTA_won_categories','BAFTA_nominated_categories','Golden_Globes_won_categories','Golden_Globes_nominated_categories','Oscar_nominated_categories','genre','year','release_date','release_date.year', 'release_date.month', 'release_date.day-of-month', 'release_date.day-of-week']
drop_cols = drop_cols + ord_cols
df.drop(columns = drop_cols, inplace = True)
df_pp = pd.concat([df,categorical_data,y], axis=1)
df_pp.shape

#trying to figure out mcar
df_mcar = pd.concat([df,df_mcar,y], axis=1)
mcar_test(df_mcar)

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1404: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


nan

In [4]:
df_pp.to_csv('/home/jovyan/data1030/data1030-oscars-prediction-project/data/pre_training_data.csv', index=False)

In [5]:
df_pp['Oscar_Best_Picture_won'].value_counts()

0    1217
1      18
Name: Oscar_Best_Picture_won, dtype: int64

In [6]:
import numpy as np
import pandas as pd
import math as ma
import scipy.stats as st

def checks_input_mcar_tests(data):
    """ Checks whether the input parameter of class McarTests is correct
            Parameters
            ----------
            data:
                The input of McarTests specified as 'data'
            Returns
            -------
            bool
                True if input is correct
            """

    if not isinstance(data, pd.DataFrame):
        print("Error: Data should be a Pandas DataFrame")
        return False

    if not any(data.dtypes.values == np.float):
        if not any(data.dtypes.values == np.int):
            print("Error: Dataset cannot contain other value types than floats and/or integers")
            return False

    if not data.isnull().values.any():
        print("Error: No NaN's in given data")
        return False

    return True


def mcar_test(data):
    """ Implementation of Little's MCAR test
    Parameters
    ----------
    data: Pandas DataFrame
        An incomplete dataset with samples as index and variables as columns
    Returns
    -------
    p_value: Float
        This value is the outcome of a chi-square statistical test, testing whether the null hypothesis
        'the missingness mechanism of the incomplete dataset is MCAR' can be rejected.
    """

    if not checks_input_mcar_tests(data):
        raise Exception("Input not correct")

    dataset = data.copy()
    vars = dataset.dtypes.index.values
    n_var = dataset.shape[1]

    # mean and covariance estimates
    # ideally, this is done with a maximum likelihood estimator
    gmean = dataset.mean()
    gcov = dataset.cov()

    # set up missing data patterns
    r = 1 * dataset.isnull()
    mdp = np.dot(r, list(map(lambda x: ma.pow(2, x), range(n_var))))
    sorted_mdp = sorted(np.unique(mdp))
    n_pat = len(sorted_mdp)
    correct_mdp = list(map(lambda x: sorted_mdp.index(x), mdp))
    dataset['mdp'] = pd.Series(correct_mdp, index=dataset.index)

    # calculate statistic and df
    pj = 0
    d2 = 0
    for i in range(n_pat):
        dataset_temp = dataset.loc[dataset['mdp'] == i, vars]
        select_vars = ~dataset_temp.isnull().any()
        pj += np.sum(select_vars)
        select_vars = vars[select_vars]
        means = dataset_temp[select_vars].mean() - gmean[select_vars]
        select_cov = gcov.loc[select_vars, select_vars]
        mj = len(dataset_temp)
        parta = np.dot(means.T, np.linalg.solve(select_cov, np.identity(select_cov.shape[1])))
        d2 += mj * (np.dot(parta, means))

    df = pj - n_var

    # perform test and save output
    p_value = 1 - st.chi2.cdf(d2, df)

    return p_value


In [11]:
mcar_test(df_pp[df_pp.columns[df_pp.isnull().any(axis=0)]])

0.0

In [43]:
df_pp.loc[:,(df_pp == 0).any(axis=0)];

In [46]:
# trying to get all columns that aren't one-hot-encode columns
mcar_test(df_pp.loc[:,set(df.columns).difference(set((df_pp == 0).any(axis=0)))])

nan